In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time_templates.utilities.plot import plot_profile_1d, plot_hist
import pandas as pd
from time_templates.datareader.get_data import fetch_MC_data_from_tree
from names import DICT_COMP_LABELS
colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']
markers = ['o', 'x', 'v', '<', '^', '>', 's']
lss = ['-', '--', ':', '-.']
from matplotlib.colors import LogNorm

In [ ]:
df = pd.read_pickle('/home/mart/auger/data/time_templates/mean_df/df_means_merged.pl')
# df = df.loc[[0.9, 0.8, 0.7, 0.6, 0.5]]
# df = fetch_MC_data_from_tree(energy='19_19.5', primary='proton', dense=True, no_traces=True)
# df.index.get_level_values(level=0).unique()
# df = df.query('MCr < 2100')

In [ ]:
df.query('nstations > 10')

In [ ]:
df = pd.read_pickle('/home/mart/auger/data/time_templates/binned_df/binned_df_EPOS_LHC_proton_19_19.5_phased_traces.pl')

In [ ]:
df_gb = df.groupby(df.index.names)
df_mean = df_gb.mean()
df_var = df_gb.var()
n = df_gb["MClgE"].count()
df_mean['nstations'] = n
for key in df_var.keys():
    df_mean[key + "_var"] = df_var[key]

In [ ]:
def fourier_expansion_3deg(psi, p):
    return (p[0] * np.cos(0) + p[1]*np.sin(psi) +
                    p[2] * np.cos(psi) + p[3]*np.sin(2*psi) + 
                    p[4]*np.cos(2*psi) + p[5]*np.sin(3*psi))

In [ ]:
ct2 = 0.4
DX = 500
df_ = df_mean.loc[ct2].loc[DX]
comp = 'em'
r_unique = np.sort(df_.index.get_level_values(level=0).unique())
cos_psi_unique = np.sort(df_.index.get_level_values(level=1).unique())
nr = len(r_unique)
ncp = len(cos_psi_unique)
y = df_[f'rho_{comp}_MC'].values
yerr= np.sqrt(df_[f'rho_{comp}_MC_var']/df_['nstations']).values
r = df_['MCr'].values
cos_psi = df_['MCcospsi'].values
n = df_['nstations'].values

mask = (n > 3) & (yerr > 0) & np.isfinite(y*yerr*n)
from scipy.optimize import minimize

def fit_func(r, cos_psi, p):
    lgr = np.log10(r/1000)
    psi = np.arccos(cos_psi)
    return (fourier_expansion_3deg(psi, p[:6])*lgr +
            fourier_expansion_3deg(psi, p[6:12])*lgr**2 +
            fourier_expansion_3deg(psi, p[12:])*lgr**3 )

lgy = np.log(y+1e-7)
lgyerr = yerr/y
minfunc = lambda x: np.sum((lgy[mask]-fit_func(r[mask], cos_psi[mask], x))**2/lgyerr[mask]**2)
res = minimize(minfunc, x0=np.ones(18), method='Nelder-Mead')

res = minimize(minfunc, x0=res['x'], method='Nelder-Mead')
res = minimize(minfunc, x0=res['x'], method='L-BFGS-B')

res, len(lgy[mask])-19

In [ ]:
%matplotlib notebook

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='3d')

ax.errorbar(r, cos_psi, y, zerr=yerr, marker='o', ls='', color='k')

X, Y = np.meshgrid(np.linspace(500, 2500), np.linspace(-1, 1))
Z = np.exp(fit_func(X, Y, res['x']))
surf = ax.plot_surface(X, Y, Z, norm=LogNorm(1e-4, 30), rstride=1, cstride=1,
                cmap='viridis')
# ax.set_zlim([1e-1, 20])
# ax.set_zscale('log')

fig.colorbar(surf)

In [ ]:
ct2s = df.index.get_level_values(level=0).unique()
DXs = df.index.get_level_values(level=1).unique()
rs = df.index.get_level_values(level=2).unique()
cps = df.index.get_level_values(level=3).unique()

In [ ]:
df = df.drop([key for key in df.keys() if 'trace' in key], axis=1)

In [ ]:
df['MCDX_bin'] = pd.cut(df['MCDXstation'], np.arange(0, 1050, 50))

In [ ]:
df_gb = df.groupby(['MCDX_bin', 'MCr_round_idx'])
df_sum = df_gb.sum()
count = df_gb.count()
df_mean = df_sum/count
for key in df_mean.keys():
    if 'var' in key:
        df_mean[key] = df_sum[key]
df_mean['nstations'] = df_sum['nstations']

In [ ]:
df

In [ ]:
from time_templates.utilities.fitting import plot_fit_curve, fit_curve

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(15, 15))

#TODO check with signals and divide by y[3]; should give same answers
def lgLDF(x, a, b):
    return a*(x-3) + b*(x-3)**2

from collections import defaultdict
dd = defaultdict(list)

NSTATION_CUT = 5
R_MAX_CUT = 2600
comp_labels = list(DICT_COMP_LABELS.keys())
for comp, ax in zip(comp_labels, axes.flatten()):
    for DX_bin in df_mean.index.get_level_values(level=0).unique():
        DX = DX_bin.mid
        df_bin = df_mean.loc[DX, :]
        try:
            rho1000 = df_bin.loc[1000][f'rho_{comp}_MC']
        except KeyError:
            continue
        x = df_bin['MCr'].values
        y = df_bin[f'rho_{comp}_MC'].values
        y = y/rho1000
        n = df_bin['nstations'].values
        yerr = np.sqrt(df_bin[f'rho_{comp}_MC_var'].values/n)
        lgx = np.log10(x)
        lgy = np.log10(y)
        lgyerr = yerr/y * 1/np.log(10)
        mask = np.isfinite(lgx*lgy*lgyerr) & (lgyerr > 0) & (n > NSTATION_CUT) & (x < R_MAX_CUT)
        if len(lgx[mask]) < 4:
            continue
        else:
            ax, (fitp, fitp_err, chi2, ndof) = plot_fit_curve(lgx[mask], lgy[mask], lgLDF, yerr=lgyerr[mask], p0=[5, -1], ax=ax, smoother_x=True)
        ax.axhline(0, ls='--', color='k')
        ax.axvline(3, ls='--', color='k')
        dd['DX'].append(df_bin['MCDXstation'].mean())
        dd['comp'].append(comp)
        dd['chi2'].append(chi2)
        dd['ndof'].append(ndof)
        for i, p in enumerate(fitp):
            dd[f'p{i}'].append(p)
            dd[f'perr{i}'].append(fitp_err[i])

df_fit = pd.DataFrame(dd)

In [ ]:
df_fit['red_chi2'] = df_fit['chi2']/df_fit['ndof']

In [ ]:
DX0 = 400
def p_DX_lin(DX, a, b, c):
    x = DX/DX0 - 1
    return a + b * x + c*x**2

dict_ab_fit = {}
for comp in comp_labels:
    f, axes = plt.subplots(1, 2, figsize=(12, 5))
    dict_ab_fit[comp] = {}
    for ip, ax in enumerate(axes):
        df_ =  df_fit.query(f'comp == "{comp}" & ndof > 2 & red_chi2 < 10 & DX < 1000')
        ax, (pfit, pfit_err, chi2, ndof) = plot_fit_curve(df_['DX'], df_[f'p{ip}'], func=p_DX_lin, yerr=df_[f'perr{ip}']*np.sqrt(df_['red_chi2']), ax=ax)
        dict_ab_fit[comp][ip] = pfit
        ax.set_title(comp + f' {ip}')
        ax.legend()

In [ ]:
for comp in comp_labels:
    a = p_DX_lin(df['MCDXstation'].values, *dict_ab_fit[comp][0])
    b = p_DX_lin(df['MCDXstation'].values, *dict_ab_fit[comp][1])    

    lgr = np.log10(df['MCr'].values)
    df[f'rho_{comp}_pred'] = 10**lgLDF(lgr, a, b)
    df[f'rho_{comp}_residual'] = df[f'rho_{comp}_MC']/df[f'rho_{comp}_pred']
    df[f'rho_{comp}_residual_err'] = df[f'rho_{comp}_residual'] * np.sqrt(df[f'rho_{comp}_MC_var']/df['nstations']/df[f'rho_{comp}_pred']**2 + 0.01**2)

In [ ]:
comp = 'em'
plot_hist(df[f'rho_{comp}_residual'], bins=np.linspace(0.5, 1.5));

In [ ]:
def assym_correction(a, DX, r, sintheta, cospsi):
    return np.exp(a*DX/DX0 * r/1000 * sintheta*cospsi)

f, ax = plt.subplots(1)

dd = defaultdict(list)
for comp in comp_labels:
    print("At", comp)
    for ct2 in ct2s:
        ct2 = ct2.mid
        sintheta = np.sqrt(1-ct2)
        for r in rs:
            for cp in cps:
                df_bin = df.loc[ct2, :, r, cp]
                x = df_bin['MCDXstation']
                y = df_bin[f'rho_{comp}_residual']
                yerr = df_bin[f'rho_{comp}_residual_err']
                nstations = df_bin['nstations']
                mask = np.isfinite(x*y*yerr) & (yerr > 0) & (nstations > 10)
                if len(x[mask]) < 2:
                    continue
                fitp, fitp_err, chi2, ndof = fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: assym_correction(a, x, r, sintheta, cp))
#                 print(fitp)
                dd['comp'].append(comp)
                dd['ct2'].append(ct2)
                dd['r'].append(r)
                dd['cp'].append(cp)
                dd['afit'].append(fitp[0])
                dd['afit_err'].append(fitp_err[0])
                if comp == 'em' and r == 1200 and ct2 < 0.5:
                    plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a: assym_correction(a, x, r, sintheta, cp), ax=ax)

# ax.legend(fontsize=8, ncol=3)    
df_assym = pd.DataFrame(dd)

In [ ]:
dict_assym_afit = df_assym.groupby('comp').mean()['afit']
dict_assym_afit['em']

In [ ]:
for comp in comp_labels:
    a = p_DX_lin(df['MCDXstation'].values, *dict_ab_fit[comp][0])
    b = p_DX_lin(df['MCDXstation'].values, *dict_ab_fit[comp][1])    
    lgr = np.log10(df['MCr'].values)
    df[f'rho_{comp}_pred'] = 10**lgLDF(lgr, a, b) * assym_correction(dict_assym_afit[comp], df['MCDXstation'], df['MCr'], np.sin(df['MCTheta']), df['MCcospsi'])
    df[f'rho_{comp}_residual'] = df[f'rho_{comp}_MC']/df[f'rho_{comp}_pred']
    df[f'rho_{comp}_residual_err'] = df[f'rho_{comp}_residual'] * np.sqrt(df[f'rho_{comp}_MC_var']/df['nstations']/df[f'rho_{comp}_pred']**2 + 0.01**2)

In [ ]:
comp = 'em'
plot_hist(df[f'rho_{comp}_residual'], bins=np.linspace(0.5, 1.5));

In [ ]:
comp = 'em'
plot_profile_1d(df['MCcospsi'], df[f'rho_{comp}_residual'], bins=20);